## ■ 틱텍토 게임의 구성 요소

1. 환경클래스
2. 인간 플레이어 클래스
3. AI 플레이어 클래스
    - 몬테카를로(P.265)
    - 살사
    - 큐러닝
    - DQN

## ■ 몬테카를로 클래스

1. init 함수 : num_playout = 1000 # 플레이 아웃할 횟수
2. select_action 함수 : 현재 상태에서 어느곳에 수를 둘지 수를 출력하는 함수
    - 남은 수들 중에 둘수 있는 곳이 어딘지 가능한 행동조사
    - 플레이 아웃을 1000번 수행, 남은수 자리에 승리한 횟수를 넣는다. (가치함수)
    - 남은 자리중에 가치가 가장 높은곳에 수를 둔다.
    
    
3. playout 함수 : 현재상태에서 끝까지 게임을 해서 승리, 패, 비김이 될때까지 게임을 진행하는 함수

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import copy
import math
from tqdm import tqdm

class Environment():
    
    def __init__(self):
    # 보드는 0으로 초기화된 9개의 배열로 준비
    # 게임종료 : done = True
        self.board_a = np.zeros(9)
        self.done = False
        self.reward = 0
        self.winner = 0
        self.print = False

    def move(self, p1, p2, player):
    # 각 플레이어가 선택한 행동을 표시 하고 게임 상태(진행 또는 종료)를 판단
    # p1 = 1, p2 = -1로 정의
    # 각 플레이어는 행동을 선택하는 select_action 메서드를 가짐
        if player == 1:
            pos = p1.select_action(env, player)
        else:
            pos = p2.select_action(env, player)
        
        # 보드에 플레이어의 선택을 표시
        self.board_a[pos] = player
        if self.print:
            print(player)
            self.print_board()
        # 게임이 종료상태인지 아닌지를 판단
        self.end_check(player)
        
        return  self.reward, self.done
 
    # 현재 보드 상태에서 가능한 행동(둘 수 있는 장소)을 탐색하고 리스트로 반환
    def get_action(self):
        observation = []
        for i in range(9):
            if self.board_a[i] == 0:
                observation.append(i)
        return observation
    
    # 게임이 종료(승패 또는 비김)됐는지 판단
    def end_check(self,player):
        # 0 1 2
        # 3 4 5
        # 6 7 8
        # 승패 조건은 가로, 세로, 대각선 이 -1 이나 1 로 동일할 때 
        end_condition = ((0,1,2),(3,4,5),(6,7,8),(0,3,6),(1,4,7),(2,5,8),(0,4,8),(2,4,6))
        for line in end_condition:
            if self.board_a[line[0]] == self.board_a[line[1]] \
                and self.board_a[line[1]] == self.board_a[line[2]] \
                and self.board_a[line[0]] != 0:
                # 종료됐다면 누가 이겼는지 표시
                self.done = True
                self.reward = player
                return
        # 비긴 상태는 더는 보드에 빈 공간이 없을때
        observation = self.get_action()
        if (len(observation)) == 0:
            self.done = True
            self.reward = 0            
        return
        
    # 현재 보드의 상태를 표시 p1 = O, p2 = X    
    def print_board(self):
        print("+----+----+----+")
        for i in range(3):
            for j in range(3):
                if self.board_a[3*i+j] == 1:
                    print("|  O",end=" ")
                elif self.board_a[3*i+j] == -1:
                    print("|  X",end=" ")
                else:
                    print("|   ",end=" ")
            print("|")
            print("+----+----+----+")
            



class Human_player():
    
    def __init__(self):
        self.name = "Human player"
        
    def select_action(self, env, player):
        while True:
            # 가능한 행동을 조사한 후 표시
            available_action = env.get_action()
            print("possible actions = {}".format(available_action))

            # 상태 번호 표시
            print("+----+----+----+")
            print("+  0 +  1 +  2 +")
            print("+----+----+----+")
            print("+  3 +  4 +  5 +")
            print("+----+----+----+")
            print("+  6 +  7 +  8 +")
            print("+----+----+----+")
                        
            # 키보드로 가능한 행동을 입력 받음
            action = input("Select action(human) : ")
            action = int(action)
            
            # 입력받은 행동이 가능한 행동이면 반복문을 탈출
            if action in available_action:
                return action
            # 아니면 행동 입력을 반복
            else:
                print("You selected wrong action")
        return

            
class Monte_Carlo_player():
    
    def __init__(self):
        self.name = "MC player"
        self.num_playout = 1000
        
    def select_action(self, env, player):
        # 가능한 행동 조사
        available_action = env.get_action()
        V = np.zeros(len(available_action))
        
        for i in range(len(available_action)):
            # 플레이아웃을 100번 반복
            for j in range(self.num_playout):
                # 지금 상태를 복사해서 플레이 아웃에 사용
                temp_env = copy.deepcopy(env)
                # 플레이아웃의 결과는 승리 플레이어의 값으로 반환
                # p1 이 이기면 1, p2 가 이기면 -1
                self.playout(temp_env, available_action[i], player)
                if player == temp_env.reward:
                    V[i] += 1
   
        return available_action[np.argmax(V)]    

    # 플레이아웃 재귀함수
    # 게임이 종료상태 (승 또는 패 또는 비김) 가 될때까지 행동을 임의로 선택하는 것을 반복
    # 플레이어는 계속 바뀌기 때문에 (-)를 곱해서 -1, 1, -1 이 되게함    
    def playout(self, temp_env, action, player):
        
        temp_env.board_a[action] = player
        temp_env.end_check(player)
        # 게임 종료 체크
        if temp_env.done == True:
            return 
        else:
            # 플레이어 교체
            player = -player
            # 가능한 행동 조사
            available_action = temp_env.get_action()
            # 무작위로 행동을 선택
            action = np.random.randint(len(available_action))
            self.playout(temp_env, available_action[action], player)    


np.random.seed(0)

p1 = Human_player()
# p2 = Human_player()

# p1 = Random_player()
# p2 = Random_player()

# p1 = Monte_Carlo_player()
# p1.num_playout = 100
p2 = Monte_Carlo_player()
p2.num_playout = 1000

# p1 = p1_Qplayer
# p1.epsilon = 0

# p2 = p2_Qplayer
# p2.epsilon = 0

# p1 = p1_DQN
# p1.epsilon = 0

# 지정된 게임 수를 자동으로 두게 할 것인지 한게임씩 두게 할 것인지 결정
# auto = True : 지정된 판수(games)를 자동으로 진행 
# auto = False : 한판씩 진행

auto = False

# auto 모드의 게임수
games = 100

print("pl player : {}".format(p1.name))
print("p2 player : {}".format(p2.name))

# 각 플레이어의 승리 횟수를 저장
p1_score = 0
p2_score = 0
draw_score = 0


if auto: 
    # 자동 모드 실행
    for j in tqdm(range(games)):
        
        np.random.seed(j)
        env = Environment()
        
        for i in range(10000):
            # p1 과 p2가 번갈아 가면서 게임을 진행
            # p1(1) -> p2(-1) -> p1(1) -> p2(-1) ...
            reward, done = env.move(p1,p2,(-1)**i)
            # 게임 종료 체크
            if done == True:
                if reward == 1:
                    p1_score += 1
                elif reward == -1:
                    p2_score += 1
                else:
                    draw_score += 1
                break

else:                
    # 한 게임씩 진행하는 수동 모드
    np.random.seed(1)
    while True:
        
        env = Environment()
        env.print = False
        for i in range(10000):
            reward, done = env.move(p1,p2,(-1)**i)
            env.print_board()
            if done == True:
                if reward == 1:
                    print("winner is p1({})".format(p1.name))
                    p1_score += 1
                elif reward == -1:
                    print("winner is p2({})".format(p2.name))
                    p2_score += 1
                else:
                    print("draw")
                    draw_score += 1
                break
        
        # 최종 결과 출력        
        print("final result")
        env.print_board()

        # 한게임 더?최종 결과 출력 
        answer = input("More Game? (y/n)")

        if answer == 'n':
            break           

print("p1({}) = {} p2({}) = {} draw = {}".format(p1.name, p1_score,p2.name, p2_score,draw_score))

###  문제2. 플레이 아웃횟수를 줄이면 몬테카를로 틱텍토를 AI가 먼저 두는 선수여도 이길수 있는지 확인하시오!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import copy
import math
from tqdm import tqdm

class Environment():
    
    def __init__(self):
    # 보드는 0으로 초기화된 9개의 배열로 준비
    # 게임종료 : done = True
        self.board_a = np.zeros(9)
        self.done = False
        self.reward = 0
        self.winner = 0
        self.print = False

    def move(self, p1, p2, player):
    # 각 플레이어가 선택한 행동을 표시 하고 게임 상태(진행 또는 종료)를 판단
    # p1 = 1, p2 = -1로 정의
    # 각 플레이어는 행동을 선택하는 select_action 메서드를 가짐
        if player == 1:
            pos = p1.select_action(env, player)
        else:
            pos = p2.select_action(env, player)
        
        # 보드에 플레이어의 선택을 표시
        self.board_a[pos] = player
        if self.print:
            print(player)
            self.print_board()
        # 게임이 종료상태인지 아닌지를 판단
        self.end_check(player)
        
        return  self.reward, self.done
 
    # 현재 보드 상태에서 가능한 행동(둘 수 있는 장소)을 탐색하고 리스트로 반환
    def get_action(self):
        observation = []
        for i in range(9):
            if self.board_a[i] == 0:
                observation.append(i)
        return observation
    
    # 게임이 종료(승패 또는 비김)됐는지 판단
    def end_check(self,player):
        # 0 1 2
        # 3 4 5
        # 6 7 8
        # 승패 조건은 가로, 세로, 대각선 이 -1 이나 1 로 동일할 때 
        end_condition = ((0,1,2),(3,4,5),(6,7,8),(0,3,6),(1,4,7),(2,5,8),(0,4,8),(2,4,6))
        for line in end_condition:
            if self.board_a[line[0]] == self.board_a[line[1]] \
                and self.board_a[line[1]] == self.board_a[line[2]] \
                and self.board_a[line[0]] != 0:
                # 종료됐다면 누가 이겼는지 표시
                self.done = True
                self.reward = player
                return
        # 비긴 상태는 더는 보드에 빈 공간이 없을때
        observation = self.get_action()
        if (len(observation)) == 0:
            self.done = True
            self.reward = 0            
        return
        
    # 현재 보드의 상태를 표시 p1 = O, p2 = X    
    def print_board(self):
        print("+----+----+----+")
        for i in range(3):
            for j in range(3):
                if self.board_a[3*i+j] == 1:
                    print("|  O",end=" ")
                elif self.board_a[3*i+j] == -1:
                    print("|  X",end=" ")
                else:
                    print("|   ",end=" ")
            print("|")
            print("+----+----+----+")
            



class Human_player():
    
    def __init__(self):
        self.name = "Human player"
        
    def select_action(self, env, player):
        while True:
            # 가능한 행동을 조사한 후 표시
            available_action = env.get_action()
            print("possible actions = {}".format(available_action))

            # 상태 번호 표시
            print("+----+----+----+")
            print("+  0 +  1 +  2 +")
            print("+----+----+----+")
            print("+  3 +  4 +  5 +")
            print("+----+----+----+")
            print("+  6 +  7 +  8 +")
            print("+----+----+----+")
                        
            # 키보드로 가능한 행동을 입력 받음
            action = input("Select action(human) : ")
            action = int(action)
            
            # 입력받은 행동이 가능한 행동이면 반복문을 탈출
            if action in available_action:
                return action
            # 아니면 행동 입력을 반복
            else:
                print("You selected wrong action")
        return

            
class Monte_Carlo_player():
    
    def __init__(self):
        self.name = "MC player"
        self.num_playout = 5
        
    def select_action(self, env, player):
        # 가능한 행동 조사
        available_action = env.get_action()
        V = np.zeros(len(available_action))
        
        for i in range(len(available_action)):
            # 플레이아웃을 100번 반복
            for j in range(self.num_playout):
                # 지금 상태를 복사해서 플레이 아웃에 사용
                temp_env = copy.deepcopy(env)
                # 플레이아웃의 결과는 승리 플레이어의 값으로 반환
                # p1 이 이기면 1, p2 가 이기면 -1
                self.playout(temp_env, available_action[i], player)
                if player == temp_env.reward:
                    V[i] += 1
   
        return available_action[np.argmax(V)]    

    # 플레이아웃 재귀함수
    # 게임이 종료상태 (승 또는 패 또는 비김) 가 될때까지 행동을 임의로 선택하는 것을 반복
    # 플레이어는 계속 바뀌기 때문에 (-)를 곱해서 -1, 1, -1 이 되게함    
    def playout(self, temp_env, action, player):
        
        temp_env.board_a[action] = player
        temp_env.end_check(player)
        # 게임 종료 체크
        if temp_env.done == True:
            return 
        else:
            # 플레이어 교체
            player = -player
            # 가능한 행동 조사
            available_action = temp_env.get_action()
            # 무작위로 행동을 선택
            action = np.random.randint(len(available_action))
            self.playout(temp_env, available_action[action], player)    


np.random.seed(0)


# p2 = Human_player()

# p1 = Random_player()
# p2 = Random_player()

# p1 = Monte_Carlo_player()
# p1.num_playout = 100
p1 = Monte_Carlo_player()
p1.num_playout

p2 = Human_player()
# p1 = p1_Qplayer
# p1.epsilon = 0

# p2 = p2_Qplayer
# p2.epsilon = 0

# p1 = p1_DQN
# p1.epsilon = 0

# 지정된 게임 수를 자동으로 두게 할 것인지 한게임씩 두게 할 것인지 결정
# auto = True : 지정된 판수(games)를 자동으로 진행 
# auto = False : 한판씩 진행

auto = False

# auto 모드의 게임수
games = 100

print("pl player : {}".format(p1.name))
print("p2 player : {}".format(p2.name))

# 각 플레이어의 승리 횟수를 저장
p1_score = 0
p2_score = 0
draw_score = 0


if auto: 
    # 자동 모드 실행
    for j in tqdm(range(games)):
        
        np.random.seed(j)
        env = Environment()
        
        for i in range(10000):
            # p1 과 p2가 번갈아 가면서 게임을 진행
            # p1(1) -> p2(-1) -> p1(1) -> p2(-1) ...
            reward, done = env.move(p1,p2,(-1)**i)
            # 게임 종료 체크
            if done == True:
                if reward == 1:
                    p1_score += 1
                elif reward == -1:
                    p2_score += 1
                else:
                    draw_score += 1
                break

else:                
    # 한 게임씩 진행하는 수동 모드
    np.random.seed(1)
    while True:
        
        env = Environment()
        env.print = False
        for i in range(10000):
            reward, done = env.move(p1,p2,(-1)**i)
            env.print_board()
            if done == True:
                if reward == 1:
                    print("winner is p1({})".format(p1.name))
                    p1_score += 1
                elif reward == -1:
                    print("winner is p2({})".format(p2.name))
                    p2_score += 1
                else:
                    print("draw")
                    draw_score += 1
                break
        
        # 최종 결과 출력        
        print("final result")
        env.print_board()

        # 한게임 더?최종 결과 출력 
        answer = input("More Game? (y/n)")

        if answer == 'n':
            break           

print("p1({}) = {} p2({}) = {} draw = {}".format(p1.name, p1_score,p2.name, p2_score,draw_score))